# Comprehensive SMS-Aware SVR Validation Study

This notebook performs detailed validation of SMS-aware Slice-to-Volume Reconstruction across multiple experimental conditions:

- **Motion Levels:** none, mild, moderate, severe
- **SMS Ratios (mb_factor):** 1, 2, 3, 4 (sequential to high multiband)
- **Number of Stacks:** 3, 4, 5, 6, 7, 8

The study evaluates:
1. Transform averaging correctness (equality within SMS groups)
2. Reconstruction quality vs ground truth (PSNR, SSIM, NRMSE)
3. SMS vs sequential comparison at each motion level
4. Statistical significance of results

**Author:** Anand Joshi & AI Assistant  
**Date:** November 2, 2025

## 1. Import Required Libraries

In [ ]:
import os
import sys
import json
import subprocess
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from itertools import product
from scipy import stats
from datetime import datetime

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# Add project root to path
project_root = Path.cwd()
sys.path.insert(0, str(project_root))

print(f"Project root: {project_root}")
print(f"Python version: {sys.version}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

## 2. Define Experimental Parameters

Set up the parameter space for comprehensive validation.

In [ ]:
# Experimental parameters
MOTION_LEVELS = {
    'none': {'max_rot_deg': 0.0, 'max_trans_mm': 0.0, 'max_disp': 0.0},
    'mild': {'max_rot_deg': 1.0, 'max_trans_mm': 0.5, 'max_disp': 2.0},
    'moderate': {'max_rot_deg': 3.0, 'max_trans_mm': 1.0, 'max_disp': 5.0},
    'severe': {'max_rot_deg': 5.0, 'max_trans_mm': 2.0, 'max_disp': 10.0}
}

MB_FACTORS = [1, 2, 3]  # SMS ratios to test
NUM_STACKS_OPTIONS = [3, 6, 9, 12]  # Different stack counts

# Fixed parameters
GROUND_TRUTH_DIR = "test_data/ground_truths"  # Input folder with ground truth NIfTI files
OUTPUT_DIR = "test_data/sms_stacks_generated"
SLICES_PER_STACK = 20
SLICE_THICKNESS = 3  # Slice thickness in mm (set to None to auto-calculate from n_stacks)
INPLANE_RESOLUTION = 0.8  # In-plane resolution in mm

# Find all ground truth NIfTI files
ground_truth_files = sorted(Path(GROUND_TRUTH_DIR).glob("*.nii.gz"))
if not ground_truth_files:
    ground_truth_files = sorted(Path(GROUND_TRUTH_DIR).glob("*.nii"))

if len(ground_truth_files) == 0:
    raise ValueError(f"No NIfTI files found in {GROUND_TRUTH_DIR}")

# Create output directory
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

print("Stack Generation Configuration:")
print(f"  Ground truth files: {len(ground_truth_files)}")
for i, gt_file in enumerate(ground_truth_files, 1):
    print(f"    {i}. {gt_file.name}")
print(f"  Motion levels: {list(MOTION_LEVELS.keys())}")
print(f"  MB factors: {MB_FACTORS}")
print(f"  Stack counts: {NUM_STACKS_OPTIONS}")
print(f"  Slice thickness: {SLICE_THICKNESS} mm" if SLICE_THICKNESS else f"  Slice thickness: Auto-calculated from stack count")
print(f"  In-plane resolution: {INPLANE_RESOLUTION} mm")
print(f"  Total stack sets per GT: {len(MOTION_LEVELS) * len(MB_FACTORS) * len(NUM_STACKS_OPTIONS)}")
print(f"  Total stack sets: {len(ground_truth_files) * len(MOTION_LEVELS) * len(MB_FACTORS) * len(NUM_STACKS_OPTIONS)}")
print(f"  Output directory: {OUTPUT_DIR}")

# Time estimation
total_stacks = len(ground_truth_files) * len(MOTION_LEVELS) * len(MB_FACTORS) * len(NUM_STACKS_OPTIONS)
estimated_time_per_set_minutes = 5.0  # Estimated time to generate one stack set
estimated_total_hours = (total_stacks * estimated_time_per_set_minutes) / 60
print(f"\nTime Estimation:")
print(f"  Estimated time per stack set: ~{estimated_time_per_set_minutes:.1f} minutes")
print(f"  Estimated total time: ~{estimated_total_hours:.1f} hours ({estimated_total_hours*60:.0f} minutes)")
print(f"  Per ground truth: ~{estimated_total_hours/len(ground_truth_files) if len(ground_truth_files) > 0 else 0:.1f} hours")
print(f"  Note: Actual time depends on I/O speed and CPU performance")

## 3. Helper Functions

In [ ]:
def simulate_stacks(ground_truth, output_dir, num_stacks, mb_factor, motion_params, timeout=600):
    """Simulate stacks with given parameters."""
    cmd = [
        sys.executable,
        "simstack_scripts/simulate_stacks.py",
        ground_truth,
        output_dir,
        "--n-stacks", str(num_stacks),
        "--mb-factor", str(mb_factor),
        "--max-rot-deg", str(motion_params['max_rot_deg']),
        "--max-trans-mm", str(motion_params['max_trans_mm']),
        "--max-disp", str(motion_params['max_disp']),
        "--slice-thickness", str(SLICE_THICKNESS),
        "--inplane-res", str(INPLANE_RESOLUTION),
        "--noise-std", "0.02",
        "--disable-nonlinear"
    ]
    
    try:
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=timeout)
        return result.returncode == 0, result.stdout, result.stderr
    except subprocess.TimeoutExpired:
        return False, "", "Simulation timeout"

print("Helper functions defined successfully.")

## 4. Run Comprehensive Experiments

Execute all combinations of parameters and collect results.

In [ ]:
# Initialize tracking
results = []
stack_set_id = 0

# Time tracking
import time
start_time = time.time()
generation_times = []

# Calculate total stack sets
total_stack_sets = len(ground_truth_files) * len(MOTION_LEVELS) * len(MB_FACTORS) * len(NUM_STACKS_OPTIONS)

print(f"Starting stack generation...")
print(f"Total stack sets to generate: {total_stack_sets}")
print("="*80)

# Iterate through all ground truth files
for gt_idx, ground_truth_path in enumerate(ground_truth_files, 1):
    gt_name = ground_truth_path.stem  # filename without extension
    
    print(f"\n{'='*80}")
    print(f"GROUND TRUTH {gt_idx}/{len(ground_truth_files)}: {ground_truth_path.name}")
    print(f"{'='*80}")
    
    # Verify ground truth exists and is readable
    try:
        gt_img = nib.load(str(ground_truth_path))
        gt_shape = gt_img.shape
        print(f"  Loaded: shape={gt_shape}, dtype={gt_img.get_fdata().dtype}")
    except Exception as e:
        print(f"  ERROR loading ground truth: {e}")
        continue
    
    # Iterate through all parameter combinations for this ground truth
    for motion_level, mb_factor, num_stacks in product(MOTION_LEVELS.keys(), MB_FACTORS, NUM_STACKS_OPTIONS):
        stack_set_id += 1
        motion_params = MOTION_LEVELS[motion_level]
        exp_start_time = time.time()
        
        # Calculate time estimates
        if len(generation_times) > 0:
            avg_time = np.mean(generation_times)
            remaining_sets = total_stack_sets - stack_set_id + 1
            eta_seconds = avg_time * remaining_sets
            eta_hours = eta_seconds / 3600
            eta_minutes = (eta_seconds % 3600) / 60
            elapsed_hours = (time.time() - start_time) / 3600
            
            print(f"\n  Set {stack_set_id}/{total_stack_sets}: motion={motion_level}, mb={mb_factor}, n_stacks={num_stacks}")
            print(f"    Progress: {stack_set_id/total_stack_sets*100:.1f}% | Elapsed: {elapsed_hours:.1f}h | ETA: {int(eta_hours)}h {int(eta_minutes)}m")
        else:
            print(f"\n  Set {stack_set_id}/{total_stack_sets}: motion={motion_level}, mb={mb_factor}, n_stacks={num_stacks}")
            print(f"    Progress: {stack_set_id/total_stack_sets*100:.1f}% | Warming up...")
        
        # Create output directory for this stack set
        output_subdir = Path(OUTPUT_DIR) / gt_name / f"set_{stack_set_id:03d}_m{motion_level}_mb{mb_factor}_s{num_stacks}"
        
        result = {
            'set_id': stack_set_id,
            'ground_truth': gt_name,
            'gt_file': ground_truth_path.name,
            'motion_level': motion_level,
            'mb_factor': mb_factor,
            'num_stacks': num_stacks,
            'max_rot_deg': motion_params['max_rot_deg'],
            'max_trans_mm': motion_params['max_trans_mm'],
            'max_disp': motion_params['max_disp'],
            'output_dir': str(output_subdir)
        }
        
        # Generate stacks
        print("    Generating stacks...", end=" ")
        success, stdout, stderr = simulate_stacks(
            str(ground_truth_path), str(output_subdir), num_stacks, mb_factor, motion_params
        )
        
        if not success:
            print(f"FAILED")
            print(f"    Error: {stderr[:200]}")
            result['status'] = 'failed'
            result['error'] = stderr
            results.append(result)
            continue
        
        print("OK")
        
        # Verify stacks were generated
        stack_files = sorted(output_subdir.glob("sim_stack_*.nii.gz"))
        json_files = sorted(output_subdir.glob("sim_stack_*.json"))
        
        if len(stack_files) == num_stacks and len(json_files) == num_stacks:
            result['status'] = 'success'
            result['num_generated'] = len(stack_files)
            print(f"    Generated {len(stack_files)} stacks with metadata")
        else:
            result['status'] = 'incomplete'
            result['num_generated'] = len(stack_files)
            print(f"    WARNING: Expected {num_stacks} stacks, got {len(stack_files)}")
        
        results.append(result)
        
        # Track generation time
        gen_time = time.time() - exp_start_time
        generation_times.append(gen_time)
        print(f"    Completed in {gen_time:.1f} seconds")

print("\n" + "="*80)
print(f"STACK GENERATION COMPLETE")
print(f"{'='*80}")
total_time = time.time() - start_time
successful = sum(1 for r in results if r['status'] == 'success')
print(f"Total stack sets: {len(results)}")
print(f"Successful: {successful}")
print(f"Failed: {sum(1 for r in results if r['status'] == 'failed')}")
print(f"Incomplete: {sum(1 for r in results if r['status'] == 'incomplete')}")
print(f"Ground truths processed: {len(ground_truth_files)}")
print(f"Total time: {total_time/3600:.2f} hours ({total_time/60:.1f} minutes)")
if len(generation_times) > 0:
    print(f"Average time per stack set: {np.mean(generation_times):.1f} seconds")
    print(f"Total stacks generated: {successful * len(NUM_STACKS_OPTIONS) * len(MB_FACTORS) * len(MOTION_LEVELS)}")


## 5. Create Results DataFrame and Summary

In [ ]:
# Create DataFrame
df = pd.DataFrame(results)

print(f"Total stack sets: {len(df)}")
print(f"Successful: {(df['status'] == 'success').sum()}")
print(f"Failed: {(df['status'] == 'failed').sum()}")
print(f"Incomplete: {(df['status'] == 'incomplete').sum()}")
print(f"\nResults DataFrame shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

# Display statistics by ground truth
print("\n" + "="*80)
print("GENERATION STATISTICS BY GROUND TRUTH")
print("="*80)
for gt_name in sorted(df['ground_truth'].unique()):
    df_gt = df[df['ground_truth'] == gt_name]
    successful_gt = (df_gt['status'] == 'success').sum()
    print(f"\n{gt_name}:")
    print(f"  Total sets: {len(df_gt)}")
    print(f"  Successful: {successful_gt}")
    print(f"  Success rate: {successful_gt/len(df_gt)*100:.1f}%")

# Display first few rows
print("\n" + "="*80)
print("SAMPLE STACK SETS")
print("="*80)
display(df[['set_id', 'ground_truth', 'motion_level', 'mb_factor', 'num_stacks', 'status']].head(20))

# Save results
results_file = Path(OUTPUT_DIR) / "generation_results.csv"
df.to_csv(results_file, index=False)
print(f"\nResults saved to: {results_file}")

# Summary by parameters
print("\n" + "="*80)
print("SUCCESS RATE BY PARAMETERS")
print("="*80)
success_summary = df.groupby(['motion_level', 'mb_factor', 'num_stacks']).agg({
    'status': lambda x: (x == 'success').sum(),
    'set_id': 'count'
}).rename(columns={'status': 'successful', 'set_id': 'total'})
success_summary['rate'] = (success_summary['successful'] / success_summary['total'] * 100).round(1)
print(success_summary)
